In [1]:
import numpy as np
import torch

In [2]:
from models import DecisionTransformer
from environments import Map
import seaborn as sns

In [ ]:
NR_ROBOTS = 5
CONTEXT_SIZE = 150
MAP_FILE_PATH = './maps/base.txt'

map = Map(num_robots=NR_ROBOTS, context_window=CONTEXT_SIZE).load(MAP_FILE_PATH)
model = DecisionTransformer(
    context_window=CONTEXT_SIZE,
    hidden_dim=168,
    n_heads=12,
    n_decoders=6,
    reward_dim=1,
    map_dim=map.get_area(),
    path_dim=map.get_nr_robots(),
    action_dim=map.get_nr_robots()
).to('cuda')

map, model

In [ ]:
reward_states, map_states, path_states, action_states = map.get_states()
reward_states, map_states, path_states, action_states

In [ ]:
from IPython.display import clear_output
from pprint import pprint


def render_map():
    value_map = np.array(map.get_map()[:]).copy().tolist()
    targets = map.get_targets()
    robots = map.get_robots()

    for target in targets:
        y, x = target
        value_map[y][x] = 2

    for robot in robots:
        y, x = robot
        value_map[y][x] = 3

    char_map = [' ', '*', 'x', 'r']

    rendered_map = [[char_map[int(value_map[y][x])] for x in range(map.get_width())] for y in range(map.get_height())]
    compressed_map = [''.join(rendered_map[index]) for index in range(map.get_height())]
    return '\n'.join(compressed_map)


for it in range(5):
    # clear_output(wait=True)
    print(it)
    states = map.get_states()
    print(states[0])
    model_output = model(*states)
    print(model_output)
    actions = torch.round(model_output[:-1]).tolist()
    reward = model_output[-1]
    map.move_robots(actions)
    print(render_map())